## Parallel transport of Frobenius Matrices

In [13]:
def extended_euclidean_poly(f, g):
    R = f.parent()
    x = S.gens()[0]  
    r0, r1 = f, g
    A0, A1, A2 = R(1), R(0), R(0)
    B0, B1, B2 = R(0), R(1), R(0)
    
    while r1.degree() != 0:
        r2 = r0 % r1
        q = (r0-r2)/r1
        r0 = r1
        r1 = r2
    
        A2 = A0 - q*A1
        B2 = B0 - q*B1
        
        A0 = A1
        A1 = A2
        B0 = B1
        B1 = B2
        
    return (A2/r1,B2/r1)

In [14]:
def GaussManin(f, exact_bool = False):
    
    S = f.parent()
    x = S.gens()[0]  
    t = S.base_ring().gens()[0]

    f_x = f.derivative(x)
    f_t = f.derivative(t)
    
    (A,B) = extended_euclidean_poly(f, f_x)

    A_t = A.derivative(t)
    B_x = B.derivative(x)
    B_t = B.derivative(t)

    domega = -1/2*A*f_t - A_t*f + B_x*f_t-B_t*f_x
    dxomega = x*domega + B*f_t
    domega = domega.numerator()
    dxomega = dxomega.numerator()

    deg_domega = 0 if domega == 0 else domega.degree()
    deg_dxomega = max(dxomega.degree(),0)
    deg_max = max(deg_domega,deg_dxomega)
    
    vec_domega = [0] if domega == 0 else domega.coefficients(sparse = False)
    vec_domega.reverse()
    vec_domega = vector(vec_domega)
    vec_dxomega = dxomega.coefficients(sparse = False)
    vec_dxomega.reverse()
    vec_dxomega = vector(vec_dxomega)

    vec_f = f.list()
    vec_f.reverse()
    vec_f_x = f_x.list()
    vec_f_x.reverse()
    
    vec_coefx = zero_vector(S,deg_max+1)
    vec_coefx[deg_max-1] = 1
    vec_coef0 = zero_vector(S,deg_max+1)
    vec_coef0[deg_max] = 1

    exact_form = 0 #this exact form needs to be multiplied to deduce the correct exact form to subtract

    for n in range(1,deg_max):
    
        M_P = matrix(S,n,n,lambda i,j: vec_f_x[0] if i==j else vec_f_x[1] if i==j+1 else vec_f_x[2] if i==j+2 else 0 )
    
        M_Px = matrix(S,n,n,lambda i,j: (n-i-1)*vec_f[0] if i==j else (n-i)*vec_f[1] if i==j+1 else (n-i+1)*vec_f[2] if i==j+2 else (n-i+2)*vec_f[3] if i==j+3 else 0 )
        
        M = 1/2*M_P+M_Px
    
        unit_vec_xn = zero_vector(S,n)
        unit_vec_xn[0] = 1
    
        F = sum([b*x^a for a,b in enumerate(reversed(M^(-1)*unit_vec_xn))])
        exact_form += F
        
        xn_as_exact_form = 1/2*F*f_x+F.derivative(x)*f
        # xn_as_exact_form =  X^(n+1)+coeff_x*X+coeff_0
        
        list_coeff_x = xn_as_exact_form.numerator().list()
        list_coeff_x.reverse()
        coeff_x = list_coeff_x[n]
        vec_coefx[deg_max-n-1] = -1*coeff_x
    
        list_coeff_0 = xn_as_exact_form.numerator().list()
        list_coeff_0.reverse()
        coeff_0 = list_coeff_x[n+1]
        vec_coef0[deg_max-n-1] = -1*coeff_0

    xomega_coeff_domega = vec_coefx[(deg_max-deg_domega):deg_max+1].dot_product(vec_domega)
    omega_coeff_domega= vec_coef0[(deg_max-deg_domega):deg_max+1].dot_product(vec_domega)
    
    xomega_coeff_dxomega= vec_coefx.dot_product(vec_dxomega)
    omega_coeff_dxomega= vec_coef0.dot_product(vec_dxomega)
    
    GMmatrix = matrix([[omega_coeff_domega,xomega_coeff_domega],[omega_coeff_dxomega,xomega_coeff_dxomega]])(x=0)

    if exact_bool:
        return (GMmatrix,exact_form)
    else:
        return GMmatrix

In [15]:
P.<t> = PolynomialRing(QQ,'t')
Q = FractionField(P)
S.<x> = PolynomialRing(Q,'x')

N = GaussManin(x^3-t-1)

Given the matrix of Gauss Manin connection $N$, we want to compute a matrix $U$ with power series entries in $\mathbb{Q}[[t]]$ such that
$$ UN+\frac{d}{dt}U=0$$
$$ U(0) = I_n$$

In [205]:
def horizontal_basis(N,N_pow_ser = False,p_adic = False,p=5):
    
    R2.<t> = LaurentSeriesRing(QQ)
    R1.<a,b,c,d> = PolynomialRing(QQ)
    R.<t> = LaurentSeriesRing(R1,'t',default_prec = 20)
    N2 = matrix([[R(N[0,0]),R(N[0,1])],[R(N[1,0]),R(N[1,1])]])
    U = identity_matrix(R,2)
    U1 = matrix([[a,b],[c,d]])
    
    for i in range(1,R.default_prec()+1):
        U += U1*t^i
        N1 = ((U*N2+U.derivative(t))/t^(i-1))(t=0)
        J = R1.ideal(N1.list())
        U = U(a=J.reduce(a),b=J.reduce(b),c=J.reduce(c),d=J.reduce(d))

    U2 = matrix([[R2(U[0,0]),R2(U[0,1])],[R2(U[1,0]),R2(U[1,1])]])

    if p_adic:
        Q_p = Qp(p, prec = 10, type = 'capped-rel', print_mode = 'series')
        R3.<t> =LaurentSeriesRing(Q_p)
        U3 = matrix([[R3(U2[0,0]),R3(U2[0,1])],[R3(U2[1,0]),R3(U2[1,1])]])
        N3 = matrix([[R3(N2[0,0]),R3(N2[0,1])],[R3(N2[1,0]),R3(N2[1,1])]])
        
        if N_pow_ser:
            return (U3,N3)
        else:
            return U3
    else:
        if N_pow_ser:
            return (U2,N2)
        else:
            return U2

In [127]:
def reduction_coeff(f,pow_ser):
    """
    Given a power series in z with coefficients polynomials in x, uses the function reduction_deg_2g() to return the power series
    with coefficients polynomials in x fo degree <3 in the polynomial ring K[x,y,z]/(y^2-f(x),yz-1)
    """
    pow_ser_2 = pow_ser
    for i in range(len(pow_ser_2.coefficients())):
        pow_ser += -z^pow_ser_2.exponents()[i]*pow_ser_2.coefficients()[i]+z^pow_ser_2.exponents()[i]*reduction_deg_2g(f,pow_ser_2.coefficients()[i])
    return pow_ser

In [128]:
def reduction_deg_2g(f,pol):
    """
    Given a polynomial in x, returns the representative in K[x,y,z]/(y^2-f(x),yz-1) that is a polynomial in K[x][z,z^(-1)] with the 
    degree in x <=2
    """
    if pol(z=1).degree()<3:
        return pol
    else: 
        n = pol(z=1).degree()
        pol += - pol(z=1).coefficients()[-1]*(f^(floor(n/3))-1/z^(2*floor(n/3)))*x^(n%3)
        return reduction_deg_2g(f,pol)

In [129]:
def differential_zn(f,i,n):
    """
    Compute the exact differential d(x^iz^n) with i=0,1,2 and n integer
    return p(x,z) where p(x,z)dx=d(x^i z^n) and p(x,z)=\sum p_i(x)z^i with degree(p_i(x))<3 
    """
    if n == 0:
        return i*x^(i-1)*z^0
    return (i*x^(i-1)*z^n+reduction_deg_2g(f,-1/2*n*f.derivative(x)*x^i)*z^(n+2))


In [74]:
def matrix_coeff_differentials_pos(f,n):
    matrix_diff = matrix(QQ,3)
    for i in range(3):
        if differential_zn(f,i,n) == 0:
            pol_coef = 0
        else:
            pol_coef = differential_zn(f,i,n).coefficients()[-1]
        for j in range(3):
            if pol_coef == 0:
                matrix_diff[j,i] = 0
            else:
                matrix_diff[j,i] = pol_coef[j]
    return matrix_diff

def matrix_coeff_differentials_neg(f,n):
    matrix_diff = matrix(QQ,3)
    for i in range(3):
        if differential_zn(f,i,n) == 0:
            pol_coef = 0
        else:
            if i == 0:
                pol_coef = differential_zn(f,i,n-2).coefficients()[0]
            else:
                pol_coef = differential_zn(f,i,n).coefficients()[0]
        for j in range(3):
            if pol_coef == 0:
                matrix_diff[j,i] = 0
            else:
                matrix_diff[j,i] = pol_coef[j]
    return matrix_diff

def vector_coeff(pol):
    return vector([pol[0],pol[1],pol[2]])

In [105]:
def reduction_z_pos(f,pow_ser,exact_form=0):
    if pow_ser.degree()<3:
        return (pow_ser,exact_form)
    else:
        k = pow_ser.degree()
        M = matrix_coeff_differentials_pos(f,k-2)
        V = vector_coeff(pow_ser.coefficients()[-1])
        X = M.solve_right(V)
        exact_form += X.dot_product(vector([x^0*z^(k-2),x^1*z^(k-2),x^2*z^(k-2)]))
        return reduction_z_pos(f,pow_ser-X.dot_product(vector([differential_zn(f,0,k-2),differential_zn(f,1,k-2),differential_zn(f,2,k-2)])),exact_form)

In [106]:
def reduction_z_neg(f,pow_ser,exact_form=0):
    if pow_ser.exponents()[0]>0:
        return (pow_ser,exact_form)
    else:
        k = pow_ser.exponents()[0]
        M = matrix_coeff_differentials_neg(f,k)
        V = vector_coeff(pow_ser.coefficients()[0])
        X = M.solve_right(V)
        exact_form += X.dot_product(vector([x^0*z^(k-2),x^1*z^k,x^2*z^k]))
        return reduction_z_neg(f,pow_ser-X.dot_product(vector([differential_zn(f,0,k-2),differential_zn(f,1,k),differential_zn(f,2,k)])),exact_form)

In [77]:
def reduction_z(f,pow_ser):
    red_pow_ser_pos, exact_pos = reduction_z_pos(f,pow_ser)
    red_pow_ser_neg, exact_neg = reduction_z_neg(f,red_pow_ser_pos,exact_pos)

    exact_form = exact_neg + red_pow_ser_neg[1][2]/(differential_zn(f,0,-1)[1][2])*z^(-1)
    red_pow_ser_fin = red_pow_ser_neg -red_pow_ser_neg[1][2]/(differential_zn(f,0,-1)[1][2])*differential_zn(f,0,-1)

    return (red_pow_ser_fin,exact_form)

In [319]:
p = 5
Z5 = Qp(p, prec = 10, type = 'capped-rel', print_mode = 'series')
R.<x> = PowerSeriesRing(Z5)
S.<z> = PowerSeriesRing(R,default_prec=5*20)
V = FractionField(S)

def matrix_frobenius(p,f,prec=10, exact_dif = False, d_prec=20):

    R = f.parent()
    S.<z> = PowerSeriesRing(R,default_prec=p*d_prec)
    
    Z5 = Zp(p, type = 'capped-abs', print_mode = 'series')
    
    frob_x = x^p
    #frob_y = y^p*(1+(f(x^p)-f(x)^p)*1/f(x)^p)^(1/2)
    frob_z = z^p*(1+(f(x^p)-f(x)^p)*z^(2*p))^(-1/2)

    f=S(f)
    
    frob_omega = p*x^(p-1)*frob_z
    frob_eta = p*x^(2*p-1)*frob_z
    
    red_frob_eta = reduction_coeff(f,frob_eta)
    red_frob_omega = reduction_coeff(f,frob_omega)

    red_frob_eta_fin, exact_form_eta= reduction_z(f,red_frob_eta)
    red_frob_omega_fin, exact_form_omega= reduction_z(f,red_frob_omega)

    matrix_frobenius = matrix(Z5,2)
    matrix_frobenius[0,0] = red_frob_omega_fin[1][0]
    matrix_frobenius[0,1] = red_frob_omega_fin[1][1]
    matrix_frobenius[1,0] = red_frob_eta_fin[1][0]
    matrix_frobenius[1,1] = red_frob_eta_fin[1][1]

    if exact_dif:
        return (matrix_frobenius, [exact_form_omega,exact_form_eta])
    else:
        return matrix_frobenius

In [323]:
M1 = matrix_frobenius(5,x*(x-1)*(x+1))
M2 = matrix_frobenius(5,x*(x-1)*(x+1+5))

In [218]:
P.<t> = PolynomialRing(QQ,'t')
Q = FractionField(P)
S.<x> = PolynomialRing(Q,'x')

N = GaussManin(x*(x-1)*(x+1+t))

In [248]:
U,N5 = horizontal_basis(N,N_pow_ser = True, p_adic = True,p=5)
U5 = U(t=5)

A solution for the differential equation satisfied by the Frobenius Matrix and the Gauss Manin connection 
$$ NF+\frac{d}{dt}F=\frac{d\varphi(t)}{dt}A\varphi(N)$$
uses the solution of the differential equation
$$ UN+\frac{d}{dt}U=0$$
to obtain 
$$F(t)=\varphi(U)^{-1}F(0)U$$

In [332]:
F5_2 = U(t=5)^(-1)*F0*U(t=5)

In [333]:
F2.characteristic_polynomial()

(1 + O(5^8))*x^2 + (2 + O(5^8))*x + 5 + O(5^8)

In [334]:
F5_2.characteristic_polynomial()

(1 + O(5^10))*x^2 + (1 + 2*5 + 3*5^2 + 4*5^3 + 2*5^4 + 5^5 + 3*5^6 + 4*5^7 + O(5^8))*x + 2*5^3 + 3*5^4 + 5^6 + 5^7 + O(5^8)

In [294]:
R.<x> = QQ['x']
E1 = HyperellipticCurve(x*(x-1)*(x+1))
F1 = E1.matrix_of_frobenius(5,8)

E2 = HyperellipticCurve(x*(x-1)*(x+1+5))
F2 = E2.matrix_of_frobenius(5,8)

In [304]:
M0

[                       3*5^3 + 4*5^5 + 2*5^6 + 5^8 + O(5^10)                                                      O(5^20)]
[                                                     O(5^20) 4 + 2*5 + 5^2 + 2*5^3 + 5^4 + 4*5^5 + 3*5^6 + 4*5^7 + O(5^8)]

In [291]:
F1.characteristic_polynomial()

(1 + O(5^8))*x^2 + (2 + O(5^8))*x + 5 + O(5^8)